In [2]:
from pathlib import Path
import json
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import os
import re
import importlib
import inspect
import sys
import torch.nn as nn

ROOT = Path.cwd()
ROUNDS_DIR = ROOT / "Rounds"
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# ====== 설정 ======
ROUND = 1
CSV_PATH = r"C:\Users\admin\OneDrive - 중앙대학교\Federated Learning\csv\Global Model Data.csv"
feature_cols = ["year"]
target_col = "chloride"
seq_len = 10
input_size = 1  # feature_dim = 1(year)
# ==================

round_dir = ROUNDS_DIR / f"round_{ROUND:04d}"
pt_path = round_dir / "global.pt"
js_path = round_dir / "global.json"

print("round_dir:", round_dir)
print("global.pt:", pt_path.exists(), pt_path)
print("global.json:", js_path.exists(), js_path)

meta = None
if js_path.exists():
    meta = json.loads(js_path.read_text(encoding="utf-8"))
    print("\n[global.json]")
    print(json.dumps(meta, ensure_ascii=False, indent=2))

obj = torch.load(pt_path, map_location="cpu")
print("\n[torch.load] type:", type(obj))

# 추론 가능한 형태인지 판별
if hasattr(obj, "forward") and hasattr(obj, "eval"):
    model = obj
    model.eval()
    load_kind = "module"
    print("\nload_kind = module (model object stored)")
elif isinstance(obj, dict):
    # state_dict만 저장된 경우도 많음
    # 이 경우 '모델 클래스'를 알아야 forward가 가능함
    tensor_like = sum(torch.is_tensor(v) for v in obj.values())
    if tensor_like >= max(3, int(0.3 * len(obj))):
        load_kind = "state_dict"
        state_dict = obj
    else:
        # checkpoint 안에 state_dict가 있을 수 있음
        state_dict = None
        for key in ("state_dict", "model_state_dict", "model"):
            if key in obj and isinstance(obj[key], dict):
                if sum(torch.is_tensor(v) for v in obj[key].values()) >= 1:
                    state_dict = obj[key]
                    break
        load_kind = "checkpoint/state_dict" if state_dict is not None else "unknown"
    print("\nload_kind =", load_kind)
else:
    load_kind = "unknown"
    print("\nload_kind = unknown")

if load_kind != "module":
    print("\n중요: global.pt가 model 객체가 아니라 state_dict만 저장된 형태로 보입니다.")
    print("이 경우에는 모델 아키텍처(클래스)를 import해서 인스턴스를 만든 뒤 state_dict를 load해야 추론 그래프를 그릴 수 있습니다.")

round_dir: f:\OneDrive\문서\GitHub\Federated-Learning\Rounds\round_0001
global.pt: True f:\OneDrive\문서\GitHub\Federated-Learning\Rounds\round_0001\global.pt
global.json: True f:\OneDrive\문서\GitHub\Federated-Learning\Rounds\round_0001\global.json

[global.json]
{
  "round": 1,
  "model": "LSTMRegressor",
  "trained_on": "server_csv",
  "server_train_loss": 3.9048729426618936,
  "config": {
    "input_size": 1,
    "hidden_size": 64,
    "num_layers": 1,
    "output_size": 1,
    "dropout": 0.0,
    "seq_len": 10
  },
  "global_path": "Rounds\\round_0001\\global.pt"
}

[torch.load] type: <class 'dict'>

load_kind = state_dict

중요: global.pt가 model 객체가 아니라 state_dict만 저장된 형태로 보입니다.
이 경우에는 모델 아키텍처(클래스)를 import해서 인스턴스를 만든 뒤 state_dict를 load해야 추론 그래프를 그릴 수 있습니다.


In [14]:
# 1번 셀을 실행해서 state_dict가 있어야 함
if "state_dict" not in globals() or state_dict is None:
    raise RuntimeError("state_dict가 없습니다. 1번 셀을 먼저 실행하고 다시 시도하세요.")

def summarize_state_dict_safe(sd):
    rows = []
    for name, t in sd.items():
        if not torch.is_tensor(t):
            continue

        x = t.detach().cpu()
        numel = int(x.numel())
        shape = tuple(x.shape)
        dtype = str(x.dtype)

        # float 텐서만 통계 계산
        if x.is_floating_point() and numel > 0:
            # torch로 직접 계산 (numpy로 전체 변환 금지)
            mean = x.mean().item()
            std = x.std(unbiased=False).item()
            xmin = x.min().item()
            xmax = x.max().item()
            l2 = torch.linalg.vector_norm(x.reshape(-1), ord=2).item()

            nan_count = torch.isnan(x).sum().item()
            inf_count = torch.isinf(x).sum().item()

            rows.append({
                "name": name,
                "shape": str(shape),
                "dtype": dtype,
                "numel": numel,
                "mean": float(mean),
                "std": float(std),
                "min": float(xmin),
                "max": float(xmax),
                "l2_norm": float(l2),
                "nan_count": int(nan_count),
                "inf_count": int(inf_count),
            })
        else:
            rows.append({
                "name": name,
                "shape": str(shape),
                "dtype": dtype,
                "numel": numel,
                "mean": np.nan,
                "std": np.nan,
                "min": np.nan,
                "max": np.nan,
                "l2_norm": np.nan,
                "nan_count": np.nan,
                "inf_count": np.nan,
            })

    return pd.DataFrame(rows)

df = summarize_state_dict_safe(state_dict)

print("tensors:", len(df))
print("float tensors:", int(df["mean"].notna().sum()))
print("NaN sum:", int(df["nan_count"].dropna().sum()))
print("Inf sum:", int(df["inf_count"].dropna().sum()))

df.head(20)

tensors: 6
float tensors: 6
NaN sum: 0
Inf sum: 0


,name,shape,dtype,numel,mean,std,min,max,l2_norm,nan_count,inf_count
0,lstm.weight_ih_l0,"(256, 1)",torch.float32,256,0.001540,0.071490,-0.147931,0.140809,1.144102,0,0
1,lstm.weight_hh_l0,"(256, 64)",torch.float32,16384,0.000017,0.075870,-0.190784,0.191599,9.711351,0,0
2,lstm.bias_ih_l0,"(256,)",torch.float32,256,-0.006133,0.072799,-0.147553,0.136139,1.168904,0,0
3,lstm.bias_hh_l0,"(256,)",torch.float32,256,-0.005910,0.073869,-0.146652,0.153145,1.185685,0,0
4,fc.weight,"(1, 64)",torch.float32,64,-0.001603,0.067351,-0.144889,0.144173,0.538959,0,0
5,fc.bias,"(1,)",torch.float32,1,0.072429,0.000000,0.072429,0.072429,0.072429,0,0


In [15]:
if "state_dict" not in globals() or state_dict is None:
    raise RuntimeError("state_dict가 없습니다. global.pt 로드 셀을 먼저 실행하세요.")

# 옵션
PARAM_NAME = None      # 예: "fc1.weight" 처럼 정확한 키를 넣으면 그 파라미터를 그림. None이면 자동 선택(가장 큰 float 텐서)
MAX_POINTS = 20000     # 너무 크면 샘플링해서 이 개수만 그림
SEED = 0

# 1) 대상 텐서 선택
float_items = [(k, v) for k, v in state_dict.items()
               if torch.is_tensor(v) and v.is_floating_point() and v.numel() > 0]
if not float_items:
    raise RuntimeError("float 파라미터 텐서가 없습니다. global.pt 내용을 확인하세요.")

if PARAM_NAME is not None:
    if PARAM_NAME not in state_dict or not torch.is_tensor(state_dict[PARAM_NAME]):
        raise KeyError(f"PARAM_NAME='{PARAM_NAME}' 가 state_dict에 없습니다.")
    t = state_dict[PARAM_NAME]
    if (not t.is_floating_point()) or t.numel() == 0:
        raise ValueError(f"PARAM_NAME='{PARAM_NAME}' 텐서가 float이 아니거나 비어있습니다.")
    chosen_name = PARAM_NAME
else:
    chosen_name, t = max(float_items, key=lambda kv: kv[1].numel())

x_flat = t.detach().cpu().reshape(-1)
n = int(x_flat.numel())

# 2) 샘플링(메모리 안전)
rng = np.random.default_rng(SEED)
if n > MAX_POINTS:
    idx = rng.choice(n, size=MAX_POINTS, replace=False)
    idx.sort()
    xs = idx.astype(np.int64)
    ys = x_flat[idx].numpy()
else:
    xs = np.arange(n, dtype=np.int64)
    ys = x_flat.numpy()

# 3) Plotly line plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=xs, y=ys, mode="lines", name=chosen_name))

fig.update_layout(
    title=f"Global parameter x-y plot (index vs value)<br>{chosen_name} | total={n} | plotted={len(ys)}",
    xaxis_title="parameter index (flattened)",
    yaxis_title="parameter value",
    height=420,
)

fig.show()

In [16]:
# 네가 실제로 사용한 경로 그대로 넣어도 됨
CSV_PATH = r"C:\Users\admin\OneDrive - 중앙대학교\Federated Learning\csv\Global Model Data.csv"

feature_cols = ["year"]
target_col = "chloride"
seq_len = 10

df = pd.read_csv(CSV_PATH)
print(df.columns)
print(df.head())

# 필요한 컬럼만
df2 = df[feature_cols + [target_col]].dropna().reset_index(drop=True)

X = df2[feature_cols].to_numpy(dtype=np.float32)  # (N, F)
y = df2[target_col].to_numpy(dtype=np.float32)    # (N,)

def make_sequences(X, y, seq_len):
    # many-to-one: 길이 seq_len 입력 -> 마지막 시점의 y 예측
    xs, ys, xs_year = [], [], []
    for i in range(seq_len - 1, len(X)):
        xs.append(X[i - seq_len + 1:i + 1])   # (seq_len, F)
        ys.append(y[i])                       # scalar
        xs_year.append(X[i, 0])               # year (x축용: 마지막 year)
    return np.stack(xs), np.array(ys), np.array(xs_year)

X_seq, y_seq, x_year = make_sequences(X, y, seq_len)
print("X_seq:", X_seq.shape, "y_seq:", y_seq.shape, "x_year:", x_year.shape)

Index(['year', 'usage', 'snowfall', 'chloride'], dtype='object')
   year  usage  snowfall  chloride
0     9  1.731         3    3.2565
1     9  0.562         3    2.5142
2     9  0.562         3    2.3354
3     9  0.686         3    0.3078
4     9  0.090         3    0.1448
X_seq: (829, 10, 1) y_seq: (829,) x_year: (829,)


In [21]:
# nn.Module 클래스 정의가 있는 파일들을 후보로 뽑는다
patterns = [
    r"class\s+\w+\(.*nn\.Module.*\)\s*:",
    r"import\s+torch\.nn\s+as\s+nn",
    r"from\s+torch\s+import\s+nn",
    r"load_state_dict\(",
    r"def\s+(build_model|create_model|get_model|make_model)\s*\(",
]

hits = []
for root, _, files in os.walk(ROOT):
    # .git, __pycache__ 같은건 스킵
    if any(x in root for x in [".git", "__pycache__", ".ipynb_checkpoints"]):
        continue
    for f in files:
        if not f.endswith(".py"):
            continue
        p = Path(root) / f
        try:
            txt = p.read_text(encoding="utf-8", errors="ignore")
        except:
            continue
        score = 0
        for pat in patterns:
            if re.search(pat, txt):
                score += 1
        if score >= 2:  # 기준(조절 가능)
            hits.append((score, str(p.relative_to(ROOT))))

hits = sorted(hits, reverse=True)[:30]
print("Top candidate files:")
for s, fp in hits:
    print(s, fp)

# repo 루트를 import path에 추가
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

tg = importlib.import_module("train_global_and_push")

candidates = []
for name, obj in tg.__dict__.items():
    # 1) 함수 중에서 build/create/get/make_model 같은 이름 후보
    if callable(obj) and isinstance(name, str) and any(x in name.lower() for x in ["build", "create", "get", "make"]) and "model" in name.lower():
        candidates.append(("function", name, obj))
    # 2) 클래스 중 nn.Module 상속 후보
    if inspect.isclass(obj):
        try:
            import torch.nn as nn
            if issubclass(obj, nn.Module):
                candidates.append(("class", name, obj))
        except:
            pass

print("Candidates in train_global_and_push.py:")
for t, name, _ in candidates:
    print(t, name)

Top candidate files:
2 Server\model_lstm.py
2 Clients\client_update.py
Candidates in train_global_and_push.py:
class LSTMRegressor


In [3]:
def extract_state_dict(obj):
    if isinstance(obj, dict):
        tensor_like = sum(torch.is_tensor(v) for v in obj.values())
        if tensor_like >= max(3, int(0.3 * len(obj))):
            return obj
        for key in ("state_dict", "model_state_dict", "model"):
            if key in obj and isinstance(obj[key], dict):
                sd = obj[key]
                if sum(torch.is_tensor(v) for v in sd.values()) >= 1:
                    return sd
    if hasattr(obj, "state_dict") and callable(obj.state_dict):
        return obj.state_dict()
    raise RuntimeError("state_dict 추출 실패")

state_dict = extract_state_dict(obj)

# 2) hidden_size, num_layers를 state_dict에서 자동 추정
# PyTorch LSTM 파라미터 키 예: lstm.weight_ih_l0, lstm.weight_hh_l0, lstm.bias_ih_l0 ...
lstm_ih_keys = [k for k in state_dict.keys() if "weight_ih_l" in k]
lstm_hh_keys = [k for k in state_dict.keys() if "weight_hh_l" in k]

if not lstm_ih_keys or not lstm_hh_keys:
    # 모델 내부 LSTM 이름이 lstm이 아닐 수도 있음(예: rnn, encoder 등)
    # 그 경우 weight_ih_l 패턴으로 다시 찾는 게 핵심이라 위에서 이미 했고,
    # 그래도 없으면 LSTM이 아닌 모델이거나 키 패턴이 다름
    raise RuntimeError("state_dict에서 LSTM weight_ih_l / weight_hh_l 키를 찾지 못했습니다. (모델 키 네이밍 확인 필요)")

# num_layers 추정: weight_ih_l{layer} 키의 최대 layer index + 1
layer_indices = []
for k in lstm_ih_keys:
    # ...weight_ih_l0, ...weight_ih_l1 같은 부분에서 숫자 추출
    # 마지막 'l' 뒤의 숫자
    idx_str = k.split("weight_ih_l")[-1]
    # 혹시 _reverse 같은 게 붙는 bidirectional이면 l0_reverse 같은 형태도 가능
    idx_str = idx_str.split("_")[0]
    if idx_str.isdigit():
        layer_indices.append(int(idx_str))

if not layer_indices:
    raise RuntimeError("LSTM layer index 파싱 실패")

num_layers = max(layer_indices) + 1

# hidden_size 추정: weight_hh_l0 shape = (4*hidden_size, hidden_size)
# 또는 weight_ih_l0 shape = (4*hidden_size, input_size)
w_hh0_key = sorted([k for k in lstm_hh_keys if "l0" in k])[0]
w_hh0 = state_dict[w_hh0_key]
hidden_size = int(w_hh0.shape[1])

print("Inferred num_layers:", num_layers)
print("Inferred hidden_size:", hidden_size)
print("Using input_size:", input_size)

# 3) LSTMRegressor import (어디에 정의되어 있든, 이미 모듈을 알고 있으면 그걸로 import)
# 보통 train_global_and_push.py에서 LSTMRegressor를 쓰는 파일이 있을 것이라 가정.
# 가장 간단히: 프로젝트 내에서 LSTMRegressor가 정의된 모듈명을 이미 알고 있으면 여기에 넣으면 됨.
# 우선은 train_global_and_push에 LSTMRegressor가 import되어 있을 가능성이 크니 그쪽에서 가져오자.
tg = importlib.import_module("train_global_and_push")
LSTMRegressor = getattr(tg, "LSTMRegressor", None)

if LSTMRegressor is None:
    # train_global_and_push에 없으면, Server/models.py 같은 곳에 있을 수 있음
    # 네 프로젝트 구조상 "Server" 폴더가 있으니 흔한 위치를 몇 개 시도
    tried = []
    for modname in ["Server.models", "Server.model", "models", "model", "Server.lstm_model"]:
        try:
            m = importlib.import_module(modname)
            tried.append(modname)
            if hasattr(m, "LSTMRegressor"):
                LSTMRegressor = getattr(m, "LSTMRegressor")
                print("Found LSTMRegressor in:", modname)
                break
        except Exception:
            continue

if LSTMRegressor is None:
    raise RuntimeError("LSTMRegressor를 import하지 못했습니다. LSTMRegressor가 정의된 파일(모듈 경로)이 필요합니다.")

# 4) 모델 생성 + state_dict 로드 (학습 없음)
model = LSTMRegressor(input_size, hidden_size, num_layers)
missing, unexpected = model.load_state_dict(state_dict, strict=False)
model.eval()

print("load_state_dict missing keys:", len(missing))
print("load_state_dict unexpected keys:", len(unexpected))

# 5) 데이터 로드 + seq 생성
df = pd.read_csv(CSV_PATH)
df2 = df[feature_cols + [target_col]].dropna().reset_index(drop=True)

X = df2[feature_cols].to_numpy(dtype=np.float32)  # (N,1)
y = df2[target_col].to_numpy(dtype=np.float32)    # (N,)

def make_sequences(X, y, seq_len):
    xs, ys, x_year = [], [], []
    for i in range(seq_len - 1, len(X)):
        xs.append(X[i-seq_len+1:i+1])  # (seq_len,1)
        ys.append(y[i])
        x_year.append(X[i,0])
    return np.stack(xs), np.array(ys), np.array(x_year)

X_seq, y_seq, x_year = make_sequences(X, y, seq_len)

# 6) 추론 (forward만)
X_t = torch.from_numpy(X_seq)  # (B, seq_len, 1)
with torch.no_grad():
    out = model(X_t)
    if isinstance(out, (tuple, list)):
        out = out[0]
    pred = out.detach().cpu().reshape(-1).numpy()

print("pred shape:", pred.shape, "true shape:", y_seq.shape)

# 7) Plotly x-y
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_year, y=y_seq, mode="lines+markers", name="true"))
fig.add_trace(go.Scatter(x=x_year, y=pred,  mode="lines+markers", name="pred (initial global)"))
fig.update_layout(
    title=f"Initial global model inference (round {ROUND})",
    xaxis_title="year",
    yaxis_title="chloride",
    height=520,
)
fig.show()

Inferred num_layers: 1
Inferred hidden_size: 64
Using input_size: 1
load_state_dict missing keys: 0
load_state_dict unexpected keys: 0
pred shape: (829,) true shape: (829,)
